In [1]:
import multiprocessing
import time
import psutil
import os
import gc
import cv2
import numpy as np
import sklearn as skl

In [2]:
gc.enable()
dir = "/home/alice/Jupyter/OpenCV/images/"
out = "/home/alice/Jupyter/OpenCV/output/"
buoy_dir = "/home/alice/Jupyter/OpenCV/Buoys/"
num_files = len(os.listdir(dir))

buoy = cv2.imread(buoy_dir+"redbuoy.png")
buoy = cv2.cvtColor(buoy, cv2.COLOR_BGR2GRAY)


In [3]:
def img(i):
    im = cv2.imread(dir+"left"+str(i).zfill(4)+".jpg")
    im = np.array(im)[0:500][0:800]
    im = delete_blue(im)
    im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    im = cv2.GaussianBlur(im, ksize=(0,0), sigmaX=0.5, sigmaY=.75, borderType=cv2.BORDER_DEFAULT)
    im = cv2.threshold(im, 40, 255, cv2.THRESH_BINARY)[1]
    im = cv2.medianBlur(im, 3)
    im = cv2.GaussianBlur(im, ksize=(0,0), sigmaX=0.5, sigmaY=.75, borderType=cv2.BORDER_DEFAULT)

    err = str(im)
    im_nz = np.nonzero(im)
    err = "\n"+str(im_nz)
    im_nz = np.column_stack((im_nz[1], im_nz[0]))
    err = "\n"+str(im_nz)+"\n"
    
    #print(i)

    #cluster(im_nz, i)
    
    

    write(im, i)
    return [i,im_nz]

#Delete all pixels that are outside of a certain red/green range
def delete_blue(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10)))] = [0,0,0]
    return(image)

def cluster(image, index):    
    try:
        clustering = skl.DBSCAN(eps = 1, min_samples = 3)
        clustering = clustering.fit(image)
        clustering = clustering.labels_
        if index in range(1600,1650):
            print(clustering)
    except:
        return False

#for writing one instance of an image
def write(image, index):
    cv2.imwrite(out + "out" + str(index).zfill(4) + ".jpg", image)
    return True

In [4]:
if __name__ == '__main__':
    global num_files
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    starttime = time.time()
    
    results = np.array(pool.map(img,range(num_files),30))
    pool.close()
    pool.join()
    
        
print('That took {} seconds'.format(time.time() - starttime))

[[0
  array([[105, 267],
       [106, 267],
       [104, 268],
       ...,
       [ 52, 281],
       [420, 281],
       [421, 281]])]
 [1
  array([[  0, 271],
       [  0, 272],
       [  1, 272],
       [  2, 272],
       [  3, 272],
       [  4, 272],
       [  5, 272],
       [  6, 272],
       [  7, 272],
       [  8, 272],
       [  9, 272],
       [ 10, 272],
       [ 11, 272],
       [419, 272],
       [420, 272],
       [421, 272],
       [  0, 273],
       [  1, 273],
       [  2, 273],
       [  3, 273],
       [  4, 273],
       [  5, 273],
       [  6, 273],
       [  7, 273],
       [  8, 273],
       [  9, 273],
       [ 10, 273],
       [ 11, 273],
       [ 12, 273],
       [ 13, 273],
       [ 14, 273],
       [ 15, 273],
       [357, 273],
       [418, 273],
       [419, 273],
       [420, 273],
       [421, 273],
       [422, 273],
       [  0, 274],
       [  1, 274],
       [  2, 274],
       [  3, 274],
       [  4, 274],
       [  5, 274],
       [  6, 274],
     

In [5]:
print(psutil.virtual_memory().available/1000000)

10822.496256


In [6]:
F = open("results.txt","w")
for i in range(len(results)):
    F.write(str(results[i]))